In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/CNN RNN

/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/CNN RNN


In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D, Conv2D, Reshape, MaxPooling2D
from keras.datasets import imdb
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import pickle


Using TensorFlow backend.


# Preprocessing

In [0]:
def text_preprocessing(train_data,test_data):
  train_data_texts = train_data['comment']
  train_data_labels = train_data['label']
  test_data_texts = test_data['comment']
  test_data_labels = test_data['label']


  comment_texts = []
  comment_labels = []

  train_text = []
  test_text = []
  train_labels=[]
  test_labels=[]

  for label in train_data_labels:
    if label == "POSITIVE":
      train_labels.append(1)
    else:
      train_labels.append(0)
  comment_labels.append(train_labels)

  for label in test_data_labels:
    if label == "POSITIVE":
      test_labels.append(1)
    else:
      test_labels.append(0)
  comment_labels.append(test_labels)
  

  for comment in train_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    train_text.append(lines)
  comment_texts.append(train_text)

  for comment in test_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    test_text.append(lines)
  comment_texts.append(test_text)


  return comment_texts,comment_labels

In [0]:
DataTagged = pd.read_csv("/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/corpus/analyzed/comments_tagged_remove_all_punc_keep_question.csv", ";")
train_data, test_data = train_test_split(DataTagged, test_size=0.2, random_state=0)
comment_texts, comment_labels = text_preprocessing(train_data,test_data)

In [0]:
# prepare tokenizer
docs = comment_texts[0]+comment_texts[1]
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
print(vocab_size)

18416


In [0]:
encoded_docs = t.texts_to_sequences(docs)
maxlen = 155
padded_docs = pad_sequences(encoded_docs, maxlen=maxlen)

# Load FastText

In [0]:
keyed_fasttext_path = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/word_embedding/fasttext/fasttext_keyed_vectors/keyed.kv'
word_vectors = KeyedVectors.load(keyed_fasttext_path, mmap='r')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
embedding_matrix_path = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/CNN RNN/fasttetx_embedding_matrix'
f = open(embedding_matrix_path, 'rb')
embedding_matrix= np.array(pickle.load(f))

In [0]:
labels = comment_labels[0]+comment_labels[1]
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=0)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
max_features = embedding_matrix.shape[0]
embedding_size = 300

kernel_size = 5
filters = 64
pool_size = 2

lstm_output_size = 100

batch_size = 64
epoches = 20


In [0]:
# print('Loading data......')
# (x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_features)

Loading data......
17465344/17464789 [==============================] - 3s 0us/step


In [0]:
# print(len(X_train), 'train sequences')
# print(len(X_test),'test sequences')

4008 train sequences
1002 test sequences


In [0]:
# print('pad sequences (sample X time)')
# X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
# X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:',X_train.shape)
print('x_test shape:', X_test.shape)

x_train shape: (4008, 155)
x_test shape: (1002, 155)


In [0]:
print('Build the model.....')
model = Sequential()
model.add(Embedding(max_features,embedding_size,input_length=maxlen,weights=[embedding_matrix],trainable=False))
#model.add(Dropout(0.25))
model.add(Reshape((maxlen,embedding_size,1)))
model.add(Conv2D(filters,kernel_size,padding='valid', activation='relu',strides=1))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Reshape((-1,filters)))
model.add(LSTM(lstm_output_size))
model.add(Dense(32))
model.add(Dropout(0.5))
model.add(Dense(16))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Build the model.....
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 155, 300)          5524800   
_________________________________________________________________
reshape_1 (Reshape)          (None, 155, 300, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 151, 296, 64)      1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 148, 64)       0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 11100, 64)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (Non

In [0]:
print('Training the model....')
model.fit(X_train,y_train, batch_size=batch_size, epochs=epoches,validation_data=(X_test,y_test))
model.save_weights("model.h5")

Training the model....
Train on 4008 samples, validate on 1002 samples
Epoch 1/20
4008/4008 [==============================] - 1696s 423ms/step - loss: 0.6954 - accuracy: 0.5187 - val_loss: 0.6878 - val_accuracy: 0.6088
Epoch 2/20
4008/4008 [==============================] - 1707s 426ms/step - loss: 0.6840 - accuracy: 0.5506 - val_loss: 0.6633 - val_accuracy: 0.6178
Epoch 3/20
4008/4008 [==============================] - 1730s 432ms/step - loss: 0.6668 - accuracy: 0.5948 - val_loss: 0.6469 - val_accuracy: 0.6647
Epoch 4/20
4008/4008 [==============================] - 1718s 429ms/step - loss: 0.6526 - accuracy: 0.6223 - val_loss: 0.6161 - val_accuracy: 0.6776
Epoch 5/20
4008/4008 [==============================] - 1711s 427ms/step - loss: 0.6159 - accuracy: 0.6677 - val_loss: 0.5952 - val_accuracy: 0.6846
Epoch 6/20
4008/4008 [==============================] - 1739s 434ms/step - loss: 0.6041 - accuracy: 0.6854 - val_loss: 0.5965 - val_accuracy: 0.7046
Epoch 7/20
4008/4008 [=============

In [0]:
score, accuracy = model.evaluate(x_test,y_test, batch_size=batch_size)
print('Test score:', score)
print('Test_accuracy:', accuracy)